## Importing Modules

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_core.output_parsers import StrOutputParser,JsonOutputParser
from langchain.output_parsers import StructuredOutputParser, ResponseSchema,PydanticOutputParser
from groq import Groq
from langchain_core.prompts import PromptTemplate


import os
from typing import TypedDict,Annotated,Optional,Literal
from pydantic import BaseModel, EmailStr, Field

import warnings
warnings.filterwarnings("ignore")

load_dotenv()

True

In [2]:
model = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.5 ## this is creative parameter
    )

## Normal way

In [7]:
## 1st prompt detailed report
## 2nd prompt summary
template_1 = PromptTemplate(

    template = "Write a detailed report on {topic}",
    input_variables = ["topic"]

)

template_2 = PromptTemplate(

    template = "Write 2 line summary on the following text ./n {text}",
    input_variables = ["text"]

)

In [8]:
prompt1 = template_1.invoke({"topic":"blackhole"})
result = model.invoke(prompt1)

prompt2 = template_2.invoke({"text":result.content})
result_2 = model.invoke(prompt2)
print(result_2.content)


Black holes are regions of immense gravity from which nothing, not even light, can escape.  They form from the collapse of massive stars and have a profound impact on the structure and evolution of galaxies.  



## String Output Parser

In [11]:
parser = StrOutputParser()
chain = template_1 | model | parser | template_2 | model | parser

result = chain.invoke({"topic":"blackhole"})
print(result)

Black holes are incredibly dense objects with such strong gravity that nothing, not even light, can escape.  They form from the collapse of massive stars and have a profound impact on the evolution of galaxies and the universe itself. 





## Json Output Parser

In [ ]:
parser = JsonOutputParser()

template = PromptTemplate(
    template='Give me 5 facts about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction': parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result) ## we cannot get the output in desired format its a biggest flaw it doesnt enforce schema


{'facts': ['Black holes are regions in spacetime where gravity is so strong that nothing, not even light, can escape.', 'They are formed when massive stars collapse at the end of their life cycle.', 'Black holes are characterized by their event horizon, a boundary beyond which escape is impossible.', 'While we cannot see black holes directly, we can detect them through their gravitational effects on nearby objects.', 'There are different types of black holes, including stellar-mass black holes, supermassive black holes, and intermediate-mass black holes.']}


## Structured Output Parser

In [17]:
## we are creating a Schema Object
schema = [
    ResponseSchema(name='fact_1', description='Fact 1 about the topic'),
    ResponseSchema(name='fact_2', description='Fact 2 about the topic'),
    ResponseSchema(name='fact_3', description='Fact 3 about the topic'),
]

parser = StructuredOutputParser.from_response_schemas(schema) ## we are passing the schema to parser


In [ ]:
template = PromptTemplate(
    template='Give 3 fact about {topic} \n {format_instruction}',
    input_variables=['topic'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

result = chain.invoke({'topic':'black hole'})

print(result) ## u can see the keys are fact_1,fact_2 and fact_3 but cant do data validation

{'fact_1': 'Black holes are regions in spacetime where gravity is so strong that nothing, not even light, can escape.', 'fact_2': 'Black holes form when massive stars collapse at the end of their life cycle.', 'fact_3': 'Although black holes are invisible, their presence can be detected by observing the effects they have on nearby matter and light.'}


## Pydantic Parser

In [20]:
from operator import gt


class Person(BaseModel):
    name:str = Field(description="name of the person"),
    age : int = Field(gt=18,description="Age of the person",default=18),
    city : str = Field(description="Name of the city the person belongs to ")

parser = PydanticOutputParser(pydantic_object = Person) ## pass the schema here




In [24]:
template = PromptTemplate(
    template='Generate the name, age and city of a fictional {place} person \n {format_instruction}',
    input_variables=['place'],
    partial_variables={'format_instruction':parser.get_format_instructions()}
)

chain = template | model | parser

final_result = chain.invoke({'place':'India'})

print(final_result)

print("--------------------------")
final_result = chain.invoke({'place':'North Korea'})

print(final_result)

name='Anika Sharma' age=28 city='Mumbai'
--------------------------
name='Kim Mi-sun' age=28 city='Pyongyang'
